* Реализовать функцию свертки (специфицировать размер и количество фильтров, входной тензор, stride, ...)
$$O[m][x][y] = \sum_{i=0}^{R-1}\sum_{j=0}^{S-1}\sum_{k=0}^{C-1}I[k][x+i][y+j] * W[m][k][i][j]$$

* Написать фунцию реализующую сверточный слой через im2col. Сделать проверку результата с помощью прямой реализации свертки.

* Специфицировать и написать функцию реализующиую Depthwise-separable свертку.

### Прямая реализация свертки

In [1]:
import numpy as np
import torch
from tqdm import tqdm

def convolution(input_tensor, filters, stride=(1, 1)):
    batch_size, C, H, W = input_tensor.shape
    
    M, _, R, S = filters.shape
    
    stride_h, stride_w = stride
    
    OH = (H - R) // stride_h + 1
    OW = (W - S) // stride_w + 1
    
    output = np.zeros((batch_size, M, OH, OW))
    
    for b in range(batch_size):
        for m in range(M):
            for oh in range(OH):
                for ow in range(OW):
                    for i in range(R):
                        for j in range(S):
                            for c in range(C):
                                output[b, m, oh, ow] += input_tensor[b, c, oh * stride_h + i, ow * stride_w + j] * filters[m, c, i, j]
    
    return output

### Свертка im2col

In [2]:
def im2col(input_data, filter_h, filter_w, stride=(1, 1), pad=0):
    N, C, H, W = input_data.shape
    stride_h, stride_w = stride
    out_h = (H + 2 * pad - filter_h) // stride_h + 1
    out_w = (W + 2 * pad - filter_w) // stride_w + 1

    img = np.pad(input_data, [(0,0), (0,0), (pad, pad), (pad, pad)], 'constant')
    col = np.zeros((N, C, filter_h, filter_w, out_h, out_w))

    for y in range(filter_h):
        y_max = y + stride_h * out_h
        for x in range(filter_w):
            x_max = x + stride_w*out_w
            col[:, :, y, x, :, :] = img[:, :, y:y_max:stride_h, x:x_max:stride_w]

    col = col.transpose(0, 4, 5, 1, 2, 3).reshape(N*out_h*out_w, -1)
    return col


def convolution_im2col(input_tensor, filter, stride=(1, 1), pad=0):
    FN, C, FH, FW = filter.shape
    N, C, H, W = input_tensor.shape
    stride_h, stride_w = stride
    out_h = 1 + int((H + 2 * pad - FH) / stride_h)
    out_w = 1 + int((W + 2 * pad - FW) / stride_w)

    col = im2col(input_tensor, FH, FW, stride, pad)
    col_W = filter.reshape(FN, -1).T
    out = np.dot(col, col_W)

    out = out.reshape(N, out_h, out_w, -1).transpose(0, 3, 1, 2)

    return out

### Тестирование

In [3]:
def compare_convolution(input_tensor, filters, stride=(1, 1), verbose=False):
    # default convolution
    conv = convolution(input_tensor, filters, stride)

    # im2col convoluion
    conv_im2col = convolution_im2col(input_tensor, filters, stride)
    
    # torch result
    conv_torch = torch.nn.functional.conv2d(torch.Tensor(input_tensor), torch.Tensor(filters), stride=stride)
    
    if verbose:
        print("Custom Convolution Output:")
        print(conv.shape)
        print("\nTensorFlow Convolution Output:")
        print(conv_torch.shape)
        print("\nAre the outputs equal?")
        print(np.allclose(conv, conv_torch))
    return np.allclose(conv, conv_torch) and np.allclose(conv_im2col, conv)


In [4]:
def run_tests():
    for stride_h in tqdm([1, 2 ,3]):
        for stride_w in [1,2,3]:
            for batch_size in [1,2,3]:
                for channels in [1,2,3]:
                    for output_channels in [1,2,3]:
                        size = 100
                        for filter_size in [3, 5, 7]:
                            input_tensor = np.random.rand(batch_size, channels, size, size) 
                            filters = np.random.rand(output_channels, channels, filter_size, filter_size)
                            if not compare_convolution(input_tensor, filters, (stride_h, stride_w)):
                                print("Test failed!")
                                print(stride_h, stride_w, batch_size, channels, output_channels, size, filter_size)
                                return False
    return True

In [5]:
print(f'Is test passed: {run_tests()}')

100%|██████████| 3/3 [03:36<00:00, 72.07s/it] 

Is test passed: True


Тесты прошли успешно и функции свертки работают корректно. Для сравнения в качестве правильно вычесленной свертки использовалась реализация из библиотеки torch